Author: Carlos Roberto de Melo

Date: 05/27/2020

Obj: Perform the modeling of spectrum based on pPXF method by Cappellari,and obtein the kinematics./Realizar o modelamento de espectros baeado no código pPXF de Cappellari, e obter a cinemática.

In [1]:
import glob
from os import path

from astropy.io import fits
from scipy import ndimage
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import matplotlib.pyplot as plt

Adaptation of the Ppxf code to apply different spectra.
**x** and **y** are positions in px, which serves as counter and also index to open the spectrum files, which were saved in the format **y_px X x_px.fits**.

As discussed in the README, the .fits files containing the spectrum have three extensions. Note that pyton, by default, starts a count of zero.

It should be viewed as units of wavelength and flux. By default, pPXF works with wavelength in Angstrons, as we are using the MILES star library. In our case, as we normalize the spectrum, as units of intensity should not cause problems.



Adapatação do código Ppxf para se aplicar a diversos espectros.
x e y são as posições em px, de modo que servem de contador e também de index para abrir os arquivos de espectro, que foram salvos no formato **y_px X x_px.fits**.

Como discutido no README, os arquivos .fits que contém os espectros possuem três extensões. Note que o pyton, por padrão, começa a contagem do zero.

Deve-se antentar as unidades de comprimento de onda e fluxo. Por padrão, o ppxf trabalha com comprimento de onda em Angstrons pois estamos utilizando a biblioteca estelar do MILES. No nosso caso como vamos normalizar o espectro, as unidades de intensidade não devem gerar problemas.



In [23]:
ant = np.loadtxt('Output_Ppxf.txt')
ant[0:25]

array([[   0.        ,    0.        , -396.02255729,  284.25182203,
           1.66454547,   28.95856277],
       [   0.        ,    1.        , -324.41309547,  205.10599654,
           1.96541936,   20.8449083 ],
       [   0.        ,    2.        , -385.92453345,  381.85498912,
           1.66226452,   30.56212519],
       [   0.        ,    3.        , -273.81403898,  290.028026  ,
           1.45518114,   27.51162755],
       [   0.        ,    4.        , -359.94879926,  297.12493035,
           1.57159413,   22.9212819 ],
       [   0.        ,    5.        , -306.41726948,  279.77893789,
           1.53186574,   26.35084056],
       [   0.        ,    6.        , -310.27536308,  247.75067135,
           1.51819038,   18.97564075],
       [   0.        ,    7.        , -290.64263526,  295.34989611,
           1.40606355,   20.15227215],
       [   0.        ,    8.        , -285.8949286 ,  306.69959497,
           1.32167415,   20.41834491],
       [   0.        ,    9.        ,

In [25]:
ppxf_dir = '/home/carlos/Desktop/ppxf' #Directory of stellar library/Diretória da biblioteca estelar

stellar_lib = '/home/carlos/Downloads/Bibliotecas Estelares/BINTABLE'
stellar_G = '/home/carlos/Downloads/Bibliotecas Estelares/G'
Indo = glob.glob(stellar_lib + '/*.fits')
FWHM_tem = 1.35
chi_dof = []
cont = 0 
for y in range(20):
    for x in range(20):
        chi2_dof = ant[cont][4]
        print(chi2_dof)
    #READING THE SPECTRUM/LENDO UM ESPECTRO
        file = ('/home/carlos/Desktop/ESO325 MUSE-ColletBinning (01-07-20)/Espectros/%dx%d.fits'%(y,x)) #Path and name of the spectrum/Caminho e nome do arquivo do espectro
        hdu = fits.open(file) #Opening the .fits file/Abrindo o arquivo.fits
        gal_lin = hdu[1].data #Read the data/Lendo os dados
        h1 = hdu[1].header    #Read the header/Lendo o header dos dados
        
    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO
        var = hdu[2].data     #Read the variance/Lendo a variância
        h_var = hdu[2].header #Read the variance header/Lendo o header da variância
        
    #LOG-REBIN THE SPECTRUM AND VARIANCE/LOGARITMIZANDO O ESPECTRO E A VARIÂNCIA
        #SPECTRUM/ESPECTRO
        FWHM_gal = 2.71  #FWHM of telescope in Angstrons/FWHM do telescópio em Angstrons
        lambRange1 = h1['CRVAL1'] + np.array([0.,h1['CDELT1']*(h1['NAXIS1']-1)]) #Generating the wavelengths/Está linha gera os comprimentos de onda
                                #pois a variável gal_lin armazena em um array apenas as intensidades, mas não os comprimentos de onda
        
        #De-redshift
        z = 0.035 # Initial estimate of the galaxy redshift
        lambRange1 = lambRange1/(1+z) # Compute approximate restframe wavelength range
        FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
        #Depois do passo acima, a galáxia passa a estar no restframe, logo é preeciso setar
        z = 0.
        
        #Log-Rebin
        galaxy, logLam1, velscale = util.log_rebin(lambRange1, gal_lin) #Log-rebin the spectrum/Logaritimizando o espectro da galáxia
        median_galaxy = np.median(galaxy) #Mean of spectrum for normalization/Média do espectro, pois vamos normalizá-lo pela média
        galaxy = galaxy/median_galaxy        #Normalizing the spectrum/Normalizando o espectro pela média
        lam = np.exp(logLam1)                #Recovering the wavelength in Ansgstrons/Recuperando o comprimento de onda em unidades do SI
        
        #VARIANCE/VARIÂNCIA
        lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])
        erro = np.sqrt(chi2_dof*var) #1-sigma error from variance/1-sigma erro da variância
        erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
        noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro        
        
     
    
    
    
    
    #READING ONE TEMPLATE FOR DETERMINE THE LENGHT/LENDO UM TEMPLATE PARA SABER O TAMANHO DO VETOR FINAL
        #The ideia is make a empty array to after complete with the informations about the templates of the stellar libary/
                    #A ideia é crirar um array vazio para que mais tarde seja preenchido com as informações de todos os templetes.
        
        
        velscale_ratio = 2
        hdu = fits.open(Indo[0])
        data = hdu[1].data
        ssp = data[0][1]
        h2 = hdu[0].header
        
    #Log_rebin TEMPLATE/LOGARITMIZANDO O TEMPLATE
        lamRange2 = np.array([data[0][0][0], data[0][0][len(data[0][0]) - 1]]) 
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates = np.empty((sspNew.size, len(Indo)))  #Array com o tamanho necessário

    #FINAL FWHM FOR GALAXY AND TEMPLATE CONVOLUTION/CALCULANDO A FWHM FINAL PARA REALIZAR A CONVOLUÇÃO DO TEMPLATE E GALÁXIA
        FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
        sigma = FWHM_dif/2.355/0.4 #New sigma PSF/Novo sigma da PSF.
    
    #READING ALL TEMPLATES/LENDO TODOS OS TEMPLATES E JÁ LOGARITMIZANDO
        for j, file in enumerate(Indo):
            hdu = fits.open(file)
            data = hdu[1].data
            ssp = data[0][1]
            ssp = ndimage.gaussian_filter1d(ssp, sigma) #Realizando a convolução
            sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
            templates[:, j] = sspNew/np.median(sspNew)
    #ADICIONANDO UM SHIFT PARA COINCIDIR OS COMPRIMENTOS DE ONDA DO ESPECTRO E DO TEMPLATE
        #Artificial shift for the galaxy and templates wavelenght may coincide.    
        #Nem sempre os comprimentos de onda do templete e do espectro coincidem, por esse motivo é dado um 
        #shift artificial ao templete, mas que após o ajuste é retirado.
        c = 299792.458 #Speed of light/Velocidade da Luz
        dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c #Shift
        goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)  #Mask bad emission lines/Máscaras para linhas de emissão
        vel = c*np.log(1 + z)
        start = [vel, 200.] #Inicial guess/Chute inicial
    
    #START OF FIT/INÍCIO DO AJUSTE
       #For each spectrum the modeling is done. After the best solution is printed 
       #(velocity, velocity dispersion, another moments). Then, a table is write with the information about the
       #pixel position, velocity, velocity dispersion, chi²/DOF and 1-sigma error. The table is saved with name
       #'Output_Ppxf.txt'.  
        
       #Para cada espectro ele realiza o ajuste e printa abaixo o nome do px em que foi realizado o ajuste,
       #bem como as infomações do melhor ajuste (qui², Velocidade, Dispersão, Demais momentos). Então cria uma
       #tabela com as informações necessárias e a salva com o nome 'Output_Ppxf.txt'.
        pp = ppxf(templates, galaxy, noise, velscale, start,
                lam = lam, plot=True, moments=2,
                  degree=6,vsyst=dv, goodpixels=goodPixels, velscale_ratio=velscale_ratio)
    
        #For more informations about the method, visit the paper of Cappellari avaible in README
        #Para mais informações sobre o método, visite o paper de Cappellari disponível no README
        
        #Plot and save the best model/Plota e salva o melhor modelo
        
        pp.plot() #Plot the fit/Plota o fit
        plt.savefig('/home/carlos/Desktop/ESO325 MUSE-ColletBinning (01-07-20)/Spec Fit/%dx%d.png'%(y,x),format='png', dpi=1200)
        plt.clf()
        print(y,x)
        print('Vel :%f.\nDisp:%f.'%(pp.sol[0],pp.sol[1]))
        print('Best-fitting redshift z:', (0.035 + 1)*(1 + pp.sol[0]/c) - 1)
        error = pp.error*np.sqrt(pp.chi2) #1-sigma error/1-sigma erro
    #ESCREVENDO UMA TABELA COM OS DADOS DE Y_PX, X_PX,  VELOCIDADE(KM/S) E DISPERSÃO(KM/S)
        newrow = [np.int(y), np.int(x), pp.sol[0], pp.sol[1], pp.chi2, error[1]]
        if y == 0 and x == 0:
            table = np.array([np.int(y),np.int(x),pp.sol[0], pp.sol[1],pp.chi2, error[1]])
        else:
            table = np.vstack([table,newrow])
        print('\n####################\n')
        cont = cont + 1

1.6645454718727086
 Best Fit:       Vel     sigma
 comp.  0:      -352       296
chi2/DOF: 1.001
method = capfit; Jac calls: 8; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 0
Vel :-351.917441.
Disp:296.259125.
Best-fitting redshift z: 0.03378504431339513

####################

1.9654193588346764
 Best Fit:       Vel     sigma
 comp.  0:      -322       206
chi2/DOF: 1
method = capfit; Jac calls: 8; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 1
Vel :-321.827422.
Disp:206.121445.
Best-fitting redshift z: 0.03388892674773958

####################

1.6622645217915497
 Best Fit:       Vel     sigma
 comp.  0:      -382       383
chi2/DOF: 1
method = capfit; Jac calls: 9; Func calls: 29; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
0 2
Vel :-382.441448.
Disp:383.343914.
Best-fitting redshift z: 0.03367966358631502

####################

1.4551811427273014
 Best Fit:       Vel     sigm

 Best Fit:       Vel     sigma
 comp.  0:      -290       265
chi2/DOF: 1
method = capfit; Jac calls: 7; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  15 / 1273
1 6
Vel :-289.921553.
Disp:265.080851.
Best-fitting redshift z: 0.03399907819771775

####################

1.4096337362831912
 Best Fit:       Vel     sigma
 comp.  0:      -308       265
chi2/DOF: 1
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  13 / 1273
1 7
Vel :-308.063765.
Disp:264.767315.
Best-fitting redshift z: 0.033936444237451235

####################

1.2909296552475138
 Best Fit:       Vel     sigma
 comp.  0:      -310       305
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
1 8
Vel :-310.376862.
Disp:305.067486.
Best-fitting redshift z: 0.0339284585267281

####################

1.3686180673557466
 Best Fit:       Vel     sigma
 comp.  0:      -33

 Best Fit:       Vel     sigma
 comp.  0:      -317       314
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
2 12
Vel :-316.516644.
Disp:314.052928.
Best-fitting redshift z: 0.0339072616140812

####################

1.2045700719894787
 Best Fit:       Vel     sigma
 comp.  0:      -338       281
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
2 13
Vel :-337.591881.
Disp:280.939427.
Best-fitting redshift z: 0.033834501710370946

####################

1.335163949362704
 Best Fit:       Vel     sigma
 comp.  0:      -355       290
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
2 14
Vel :-355.013799.
Disp:289.756743.
Best-fitting redshift z: 0.03377435448476129

####################

1.446485949104975
 Best Fit:       Vel     sigma
 comp.  0:      -367

 Best Fit:       Vel     sigma
 comp.  0:      -293       256
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  14 / 1273
3 18
Vel :-292.797058.
Disp:255.961589.
Best-fitting redshift z: 0.03398915083877729

####################

1.3140697534811054
 Best Fit:       Vel     sigma
 comp.  0:      -278       262
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
3 19
Vel :-277.787713.
Disp:262.470462.
Best-fitting redshift z: 0.0340409689268395

####################

1.4227760324186596
 Best Fit:       Vel     sigma
 comp.  0:      -354       211
chi2/DOF: 1
method = capfit; Jac calls: 7; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
4 0
Vel :-354.104658.
Disp:210.609952.
Best-fitting redshift z: 0.03377749319239842

####################

1.4688641182848925
 Best Fit:       Vel     sigma
 comp.  0:      -29

 Best Fit:       Vel     sigma
 comp.  0:      -315       279
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  17 / 1273
5 4
Vel :-314.617642.
Disp:278.882580.
Best-fitting redshift z: 0.033913817707417415

####################

1.1304749033236483
 Best Fit:       Vel     sigma
 comp.  0:      -340       295
chi2/DOF: 1
method = capfit; Jac calls: 7; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
5 5
Vel :-339.552502.
Disp:294.810083.
Best-fitting redshift z: 0.033827732886360984

####################

0.9998585356468292
 Best Fit:       Vel     sigma
 comp.  0:      -275       317
chi2/DOF: 1
method = capfit; Jac calls: 7; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 1273
5 6
Vel :-275.466955.
Disp:316.516901.
Best-fitting redshift z: 0.034048981085093466

####################

1.1470035198539335
 Best Fit:       Vel     sigma
 comp.  0:      -

...

 Best Fit:       Vel     sigma
 comp.  0:      -309       278
chi2/DOF: 1
method = capfit; Jac calls: 7; Func calls: 26; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  13 / 1273
15 12
Vel :-308.741819.
Disp:277.761094.
Best-fitting redshift z: 0.03393410333156721

####################

1.2457034974849277
 Best Fit:       Vel     sigma
 comp.  0:      -314       267
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  16 / 1273
15 13
Vel :-314.396095.
Disp:267.042954.
Best-fitting redshift z: 0.03391458257312108

####################

1.3086763432517845
 Best Fit:       Vel     sigma
 comp.  0:      -303       299
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  15 / 1273
15 14
Vel :-303.426630.
Disp:298.885945.
Best-fitting redshift z: 0.03395245342834108

####################

1.3047312137144762
 Best Fit:       Vel     sigma
 comp.  0:  

 Best Fit:       Vel     sigma
 comp.  0:      -291       253
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  13 / 1273
16 18
Vel :-290.714822.
Disp:252.581493.
Best-fitting redshift z: 0.033996339525192765

####################

1.384203823756807
 Best Fit:       Vel     sigma
 comp.  0:      -410       351
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  16 / 1273
16 19
Vel :-410.462610.
Disp:350.511103.
Best-fitting redshift z: 0.033582923651553465

####################

1.578289405341745
 Best Fit:       Vel     sigma
 comp.  0:      -310       261
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 23; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
17 0
Vel :-309.555593.
Disp:260.901731.
Best-fitting redshift z: 0.033931293867049295

####################

1.512404136093564
 Best Fit:       Vel     sigma
 comp.  0:    

 Best Fit:       Vel     sigma
 comp.  0:      -340       228
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
18 4
Vel :-339.586660.
Disp:227.629589.
Best-fitting redshift z: 0.03382761496088049

####################

1.3808391126890047
 Best Fit:       Vel     sigma
 comp.  0:      -327       273
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 24; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
18 5
Vel :-327.099567.
Disp:273.155604.
Best-fitting redshift z: 0.03387072525280588

####################

1.6166784404918562
 Best Fit:       Vel     sigma
 comp.  0:      -329       291
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
18 6
Vel :-329.114219.
Disp:291.428987.
Best-fitting redshift z: 0.03386376989161266

####################

1.5124890735882286
 Best Fit:       Vel     sigma
 comp.  0:      -3

 Best Fit:       Vel     sigma
 comp.  0:      -272       280
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  15 / 1273
19 10
Vel :-271.864890.
Disp:280.080427.
Best-fitting redshift z: 0.03406141681257324

####################

1.443873042008896
 Best Fit:       Vel     sigma
 comp.  0:      -330       266
chi2/DOF: 1
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
19 11
Vel :-329.985438.
Disp:265.732927.
Best-fitting redshift z: 0.03386076210757749

####################

1.3648265618301896
 Best Fit:       Vel     sigma
 comp.  0:      -287       245
chi2/DOF: 1
method = capfit; Jac calls: 5; Func calls: 21; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  1 / 1273
19 12
Vel :-286.849478.
Disp:244.527968.
Best-fitting redshift z: 0.034009684194999945

####################

1.4998451770616847
 Best Fit:       Vel     sigma
 comp.  0:    

<Figure size 432x288 with 0 Axes>

In [26]:
np.savetxt('Output_Ppxf with chi2.txt',table, header= 'y_px x_px vel(km/s) disp(km/s) chi2/DOF 1-sigma', fmt='%i %i %.18e %.18e %.18e %.18e',delimiter=' ') 